# Example 2: Detecting the age-oxygen correlation

In Section 7 of [Bixel & Apai (2021)](https://ui.adsabs.harvard.edu/abs/2021AJ....161..228B/abstract) (and in [Bixel & Apai 2020](https://ui.adsabs.harvard.edu/abs/2020ApJ...896..131B/abstract)), we propose that Earth-like planets might have similar atmospheric evolution to Earth's, i.e. toward greater biogenic oxygen content over Gyr timescales. If so, this would imply a positive "age-oxygen correlation" between the fraction of Earth-like planets with atmospheric oxygen and their ages.

In this example, you will use Bioverse to determine whether a [Nautilus](https://ui.adsabs.harvard.edu/abs/2019AJ....158...83A/abstract)-like transit spectroscopy survey could test this hypothesis.

## Setup

We will begin by importing modules from Bioverse

In [1]:
# Import numpy
import numpy as np
import pandas as pd # Added for data saving
import json # Added for saving analysis results (built-in)
import pickle # Added for saving complex grid results

# Import the relevant modules
from bioverse.survey import TransitSurvey
from bioverse.generator import Generator
from bioverse.hypothesis import Hypothesis
from bioverse import analysis, plots


# Import pyplot (for making plots later) and adjust some of its settings
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['font.size'] = 20.

In this example, we will use the simulator for a transit spectroscopy survey with an effecting diameter of 50 meters.

In [2]:
generator = Generator('transit')
survey = TransitSurvey('default')

## Injecting the statistical effect

We propose that the fraction of Earth-like planets with oxygen-rich atmospheres should increase over Gyr timescales. We can simulate this in Bioverse using a function with two parameters:

- `f_life`: the fraction of exo-Earth candidates with life

- `t_half`: the "half-life" timescale over which inhabited planets become oxygenated

Let's define a function and append it to the generator.

In [3]:
def oxygen_evolution(d, f_life=0.8, t_half=3.):
    # First, assign no O2 to all planets
    d['has_O2'] = np.zeros(len(d))

    # Calculate the probability that each EEC has O2 based on its age
    EEC = d['EEC']
    P = f_life * (1 - 0.5**(d['age'][EEC]/t_half))

    # Randomly assign O2 to some EECs
    d['has_O2'][EEC] = np.random.uniform(0, 1, EEC.sum()) < P

    return d

generator.insert_step(oxygen_evolution)

Next, we can simulate a dataset and investigate the relationship between the ages and oxygen content of EECs. For now, we assume `f_life` = 80% and `t_half` = 3 Gyr.

Note that the transit survey is more readily capable of detecting ozone than O2, as it has broader and deeper absorption bands and mostly resides above the cloud deck. We assume O3 is a reliable proxy for O2.

In [4]:
# Create the planet sample
sample, detected, data = survey.quickrun(generator, f_life=0.8, t_half=3., t_total=10*365.25)

# Replaced Plotting Code with Data Saving
df = pd.DataFrame(data)
output_filename = 'simulated_raw_data_ex2.csv'
df.to_csv(output_filename, index=False)
print(f"Raw simulated data saved to {output_filename}")

Raw simulated data saved to simulated_raw_data_ex2.csv


As in Example 1, the expected trend appears to be present i.e. the presence of oxygen (via its proxy ozone) appears to correlate with age. But is this trend statistically significant?

## Defining the hypothesis

To answer this, we will define a new Hypothesis. The functional form of this hypothesis will be the same as above i.e. a two-parameter decay rate function. As in the previous example, we must specify the names of the hypothesis parameters, independent variable (`age`), and dependent variable (`has_O2`) in the function signature. We again select log-uniform prior distributions for the two parameters:

0.01 < `f_life` < 1

0.3 < `t_half` < 30 Gyr

In [5]:
def f(theta, X):
    f_life, t_half = theta
    return f_life * (1-0.5**(X/t_half))

params = ('f_life', 't_half')
features = ('age',)
labels = ('has_O2',)

bounds = np.array([[0.01, 1], [0.3, 30]])
h_age_oxygen = Hypothesis(f, bounds, params=params, features=features, labels=labels, log=(True, True))

We also need to define the null hypothesis, which states that the fraction of planets with O2 is (log-uniform) random between 0.001 to 1 and independent of age:

In [6]:
def f_null(theta, X):
    shape = (np.shape(X)[0], 1)
    return np.full(shape, theta)

bounds_null = np.array([[0.001, 1.]])
h_age_oxygen.h_null = Hypothesis(f_null, bounds_null, params=('f_O2',), features=features, labels=labels, log=(True,))

We can calculate the Bayesian evidence supporting `h_age_oxygen` in favor of `h_null` from our simulated dataset.

In [7]:
results_fit_dlnz = h_age_oxygen.fit(data)
print("The evidence in favor of the hypothesis is: dlnZ = {:.1f} (corresponds to p = {:.1E})".format(results_fit_dlnz['dlnZ'], np.exp(-results_fit_dlnz['dlnZ'])))

# Save the hypothesis fit result (dlnZ)
output_filename_fit_dlnz = 'hypothesis_fit_dlnz_result_ex2.json'

# Define a helper to serialize NumPy types
def numpy_encoder(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

with open(output_filename_fit_dlnz, 'w') as f:
    json.dump(results_fit_dlnz, f, indent=4, default=numpy_encoder)
    
print(f"Hypothesis fit (dlnZ) result saved to {output_filename_fit_dlnz}")

The evidence in favor of the hypothesis is: dlnZ = 13.5 (corresponds to p = 1.3E-06)
Hypothesis fit (dlnZ) result saved to hypothesis_fit_dlnz_result_ex2.json


The default nested sampling test reveals some compelling evidence in favor of the hypothesis, but it may not be the most efficient test for this problem.

We can also use the Mann-Whitney U test to determine whether the typical age of planets with oxygen-rich atmospheres is higher than that of oxygen-poor planets. This test should be more sensitive, as it makes no specific assumptions about the functional form of the correlation (i.e. `func` defined above is not used).

In [8]:
results_fit_mannwhitney = h_age_oxygen.fit(data, method='mannwhitney')
print('Correlation detected with p = {:.1E} significance'.format(results_fit_mannwhitney['p']))

# Save the hypothesis fit result (Mann-Whitney)
output_filename_fit_mannwhitney = 'hypothesis_fit_mannwhitney_result_ex2.json'

with open(output_filename_fit_mannwhitney, 'w') as f:
    json.dump(results_fit_mannwhitney, f, indent=4, default=numpy_encoder)
    
print(f"Hypothesis fit (Mann-Whitney) result saved to {output_filename_fit_mannwhitney}")

Correlation detected with p = 1.1E-07 significance
Hypothesis fit (Mann-Whitney) result saved to hypothesis_fit_mannwhitney_result_ex2.json


Typically, the Mann-Whitney test detects the correlation with even greater significance (i.e. lower p-values)

**Note for iPython/Jupyter users:** We will need to reload the `generator` and `h_age_oxygen` objects here to replace the ones you have defined above. These will produce the same results, but the next code block uses the `multiprocessing` module which is incompatible with functions defined in iPython.

In [9]:
generator = Generator('transit')
from bioverse.hypothesis import h_age_oxygen

## Computing statistical power

Thus far, we have assumed the fraction of planets with life to be `f_life` = 80% and `t_half` = 3 Gyr. If fewer planets are inhabited, or if the evolutionary timescale is much longer than the maximum ages probed (~10 Gyr), then the correlation will be more difficult to detect. Let's test the sensitivity of our results to both of these parameters simultaneously using the `test_hypothesis_grid` function.

Once again, we will use the more sensitive Mann-Whitney test to determine whether the correlation exists. This may take up to ten minutes - reduce `N_grid` for quicker results.

In [10]:
N_grid = 8
f_life = np.logspace(-1, 0, N_grid)
t_half = np.logspace(np.log10(0.5), np.log10(50), N_grid)
results_grid_f_life_t_half = analysis.test_hypothesis_grid(h_age_oxygen, generator, survey, method='mannwhitney', f_life=f_life, t_half=t_half, N=20, processes=8, t_total=10*365.25)

100%|██████████| 1280/1280 [08:43<00:00,  2.45it/s]


Now, we will use the `plot_power_grid()` function of the `plots` module to plot the statistical power of the survey (i.e. the fraction of tests achieving p < 0.05 significance) versus both parameters. **(The plotting code has been replaced with data saving.)**

In [11]:
# Replaced Plotting Code with Data Saving

# Save the grid results using pickle, as it contains non-JSON-serializable Hypothesis objects
output_filename_grid_1 = 'grid_results_f_life_t_half_ex2.pkl'

with open(output_filename_grid_1, 'wb') as f:
    pickle.dump(results_grid_f_life_t_half, f)

print(f"Analysis grid results (f_life vs t_half) saved to {output_filename_grid_1}")

# Also compute and save the statistical power data
power_f_life_t_half = analysis.compute_statistical_power(results_grid_f_life_t_half, method='p', threshold=0.05)

# Save inputs and 2D power array to JSON (converting arrays to lists)
power_data_grid_1 = {
    'f_life': f_life.tolist(),
    't_half': t_half.tolist(),
    'statistical_power': power_f_life_t_half.tolist()
}
output_filename_power_1 = 'statistical_power_grid_1_ex2.json'
with open(output_filename_power_1, 'w') as f:
    json.dump(power_data_grid_1, f, indent=4)
print(f"Statistical power grid data saved to {output_filename_power_1}")

Analysis grid results (f_life vs t_half) saved to grid_results_f_life_t_half_ex2.pkl
Statistical power grid data saved to statistical_power_grid_1_ex2.json


The "sweet spot" where high statistical power is achieved is where life is common (`f_life` > 50%) and the oxygenation timescale falls within ~2-10 Gyr.

We can also investigate the survey's sensitivity as a function of total survey time. Let's try values from 0.1 to 10 years, assuming `f_life` = 50% and `t_half` = 3 Gyr. This will take a few minutes.

In [12]:
t_total = np.logspace(-1, 1, 10) * 365.25
results_grid_t_total = analysis.test_hypothesis_grid(h_age_oxygen, generator, survey, method='mannwhitney', f_life=0.5, N=20, processes=8, t_total=t_total)

100%|██████████| 200/200 [01:33<00:00,  2.13it/s]


In [13]:
# Replaced Plotting Code with Data Saving

# Save the grid results using pickle
output_filename_grid_2 = 'grid_results_t_total_ex2.pkl'

with open(output_filename_grid_2, 'wb') as f:
    pickle.dump(results_grid_t_total, f)

print(f"Analysis grid results (t_total) saved to {output_filename_grid_2}")

# Compute and save the statistical power data
power_t_total = analysis.compute_statistical_power(results_grid_t_total, method='p', threshold=0.05)

power_data_grid_2 = pd.DataFrame({
    't_total': t_total,
    'mean_p': results_grid_t_total['p'].mean(axis=-1),
    'statistical_power': power_t_total
})

output_filename_power_2 = 'statistical_power_grid_2_ex2.csv'
power_data_grid_2.to_csv(output_filename_power_2, index=False)
print(f"Statistical power vs total time data saved to {output_filename_power_2}")

Analysis grid results (t_total) saved to grid_results_t_total_ex2.pkl
Statistical power vs total time data saved to statistical_power_grid_2_ex2.csv


Approximately 1-2 years will be required to detect the correlation under these assumptions.